Reconstructing virtual markers
==============================



In this tutorial, we will reconstruct virtual markers for anatomic landmarks that were not physically instrumented during the movement acquisition. We usually do this kind of reconstruction when it is not practical or feasible to stick a marker on an anatomical landmark. Instead, we track rigid bodies affixed to the whole segment, and we express the position of virtual markers relative to these rigid bodies.

This process has two steps:

1. A calibration step with several very short calibration acquisitions:

    a) A static acquisition of a few seconds where we can see every marker.

    b) Sometimes, probing acquisitions, one for each virtual marker. In each of these short acquisitions, we point the anatomical landmark using a calibrated probe. We need to see the probe's markers and the markers of the rigid body affixed to the landmark's segment.


2. An task analysis step where the rigid bodies are tracked and the virtual markers are reconstructed into the task acquisition.

In [ ]:
import kineticstoolkit.lab as ktk
import numpy as np

Read and visualize marker trajectories
--------------------------------------

We proceed exactly as in the previous tutorials, but this time we will perform the analysis based on a minimal set of markers. Let's say that for the right arm and forearm, all we have is one real marker on the lateral epicondyle, and two plates of three markers affixed to the arm and forearm segments (we will show every other in blue for easier visualization).

In [ ]:
# Read the markers
markers = ktk.kinematics.read_c3d_file(
    ktk.config.root_folder + '/data/kinematics/sample_propulsion.c3d')

# Set every unnecessary markers to blue
keep_white = ['LateralEpicondyleR', 'ArmR1', 'ArmR2', 'ArmR3',
        'ForearmR1', 'ForearmR2', 'ForearmR3']

for marker_name in markers.data:
    if marker_name not in keep_white:
        markers = markers.add_data_info(marker_name, 'Color', 'b')

# Set the point of view for 3D visualization
viewing_options = {
    'zoom': 3.5,
    'azimuth': 0.8,
    'elevation': 0.16,
    'translation': (0.2, -0.7)
}

# Create the player
player = ktk.Player(markers, **viewing_options)
player.to_html5(start_time=0, stop_time=1)

The aim of this tutorial is to reconstruct the right acromion, medial epicondyle and both styloids using static and probing acquisitions. Let's begin.

Calibration: Defining rigid body configurations using a static acquisition
--------------------------------------------------------------------------

In the static acquisition, every marker should be visible. We can use this trial to define rigid body configuration, which is how the rigid body's markers are placed into its local coordinate.

For this example, we will create arbitrary reference frames for the rigid bodies 'ArmR' and 'ForearmR'.

In [ ]:
rigid_body_definitions = dict()

# Read the static trial
markers_static = ktk.kinematics.read_c3d_file(
    ktk.config.root_folder + '/data/kinematics/sample_static.c3d')

# Show this trial, just to inspect it
player = ktk.Player(markers_static, **viewing_options)
player.to_html5(start_time=0, stop_time=0.5)

Using this trial, we now define a rigid body for the arm plate:

In [ ]:
rigid_body_definitions['ArmR'] = ktk.kinematics.define_rigid_body(
    markers_static,
    marker_names=['ArmR1', 'ArmR2', 'ArmR3'])

rigid_body_definitions['ArmR']

We proceed the same way for the forearm:

In [ ]:
rigid_body_definitions['ForearmR'] = ktk.kinematics.define_rigid_body(
    markers_static,
    marker_names=['ForearmR1', 'ForearmR2', 'ForearmR3', 'LateralEpicondyleR'])

rigid_body_definitions['ForearmR']

For the probe, we will define its rigid body manually from its known specifications. Every 6 local point is expressed relative to a reference frame that is centered at the probe's tip:

In [ ]:
rigid_body_definitions['Probe'] = {
    'Probe1': np.array(
            [[0.0021213, -0.0158328, 0.0864285, 1.0]]),
    'Probe2': np.array(
            [[0.0021213, 0.0158508, 0.0864285, 1.0]]),
    'Probe3': np.array(
            [[0.0020575, 0.0160096, 0.1309445, 1.0]]),
    'Probe4': np.array(
            [[0.0021213, 0.0161204, 0.1754395, 1.0]]),
    'Probe5': np.array(
            [[0.0017070, -0.0155780, 0.1753805, 1.0]]),
    'Probe6': np.array(
            [[0.0017762, -0.0156057, 0.1308888, 1.0]]),
}

Now that we defined these rigid body configurations, we will be able to track these rigid bodies in every other acquisition. This process can be done using the [track_rigid_body()](../api/kineticstoolkit.kinematics.track_rigid_body.rst) function.

As an example, let's see how we can track these rigid bodies in the static acquisition:

In [ ]:
# Create a target TimeSeries
rigid_bodies_static = ktk.TimeSeries()

for rigid_body_name in ['ArmR', 'ForearmR']:
    
    # Track this rigid body
    trajectory = ktk.kinematics.track_rigid_body(
        markers_static,
        local_points=rigid_body_definitions[rigid_body_name],
        label=rigid_body_name
    )
    
    # Add it to the target TimeSeries
    rigid_bodies_static.merge(trajectory, in_place=True)

player = ktk.Player(markers_static, rigid_bodies_static, **viewing_options)
player.to_html5(start_time=0, stop_time=0.5)

Calibration: Defining the virtual marker configurations based on probing acquisitions
-------------------------------------------------------------------------------------

Now we will go though every probing acquisition and apply the same process on each acquisition:

1. Locate the probe and the segment's rigid body using [track_rigid_body()](../api/kineticstoolkit.kinematics.track_rigid_body.rst);

2. Express the tip of the probe in the segment's local coordinate system;

3. Define a virtual marker based on the probe tip's local position, using [kinematics.define_virtual_marker()](../api/kineticstoolkit.kinematics.define_virtual_marker.rst).

Since this is a repetitive operation, we will create a new function that will be called for each probing acquisition:

In [ ]:
def process_probing_acquisition(file_name, rigid_body_name):

    # Load the markers
    markers_probing = ktk.kinematics.read_c3d_file(file_name)

    # Track the reference rigid body
    rigid_bodies_probing = (
        ktk.kinematics.track_rigid_body(
            markers_probing,
            local_points=rigid_body_definitions[rigid_body_name],
            label=rigid_body_name
        )
    )

    # Track the probe
    rigid_bodies_probing = rigid_bodies_probing.merge(
        ktk.kinematics.track_rigid_body(
            markers_probing,
            local_points=rigid_body_definitions['Probe'],
            label='Probe'
        )
    )
    
    # Return the local coordinates of the probed virtual marker
    return ktk.kinematics.define_local_position(
        rigid_bodies_probing,
        source_name='Probe',
        rigid_body_name=rigid_body_name
    )

Now, we can process every probing acquisition.

In [ ]:
rigid_body_definitions['ArmR']['AcromionR'] = (
    process_probing_acquisition(
        ktk.config.root_folder + '/data/kinematics/sample_probing_acromion_R.c3d',
        'ArmR'
    )
)

rigid_body_definitions['ArmR']['MedialEpicondyleR'] = (
    process_probing_acquisition(
        ktk.config.root_folder
        + '/data/kinematics/sample_probing_medial_epicondyle_R.c3d',
        'ArmR'
    )
)

rigid_body_definitions['ForearmR']['RadialStyloidR'] = (
    process_probing_acquisition(
        ktk.config.root_folder
        + '/data/kinematics/sample_probing_radial_styloid_R.c3d',
        'ForearmR'
    )
)

rigid_body_definitions['ForearmR']['UlnarStyloidR'] = (
    process_probing_acquisition(
        ktk.config.root_folder
        + '/data/kinematics/sample_probing_ulnar_styloid_R.c3d',
        'ForearmR'
    )
)

We added the virtual markers as new local points to the rigid body definitions:

In [ ]:
rigid_body_definitions['ArmR']

In [ ]:
rigid_body_definitions['ForearmR']

Task analysis: Tracking the rigid bodies
----------------------------------------

Now that we defined the rigid bodies and inluded virtual markers to it, we are ready to process the experimental trial we loaded at the beginning of this tutorial. We already loaded the markers; we will now track the rigid bodies. Since we added virtual markers to the rigid body definitions, we can use the `include_markers` option of [track_rigid_body()](../api/kineticstoolkit.kinematics.track_rigid_body.rst) to generate not only the rigid bodies' frame series, but also reconstruct every marker:

In [ ]:
rigid_bodies = ktk.TimeSeries()

for rigid_body_name in ['ArmR', 'ForearmR']:
    rigid_bodies = rigid_bodies.merge(
        ktk.kinematics.track_rigid_body(
            markers,
            local_points=rigid_body_definitions[rigid_body_name],
            label=rigid_body_name,
            include_markers = True
        )
    )

# Show those rigid bodies and markers in a player
player = ktk.Player(rigid_bodies, **viewing_options)

player.to_html5(start_time=0, stop_time=1)

That is it, we reconstructed the acromion, medial epicondyle and both styloids from probing acquisitions,  without requiring physical markers on these landmarks. We can conclude by adding the segments for clearer visualization. From now one, we could continue our analysis and calculate the elbow angles as in the previous tutorial.

In [ ]:
# Add the segments
segments = {
    'ArmR': {
        'Color': [1, 0.25, 0],
        'Links': [['AcromionR', 'MedialEpicondyleR'],
                  ['AcromionR', 'LateralEpicondyleR']]
    },
    'ForearmR': {
        'Color': [1, 0.5, 0],
        'Links': [['MedialEpicondyleR', 'RadialStyloidR'],
                  ['MedialEpicondyleR', 'UlnarStyloidR'],
                  ['LateralEpicondyleR', 'RadialStyloidR'],
                  ['LateralEpicondyleR', 'UlnarStyloidR'],
                  ['UlnarStyloidR', 'RadialStyloidR']]
    }
}

player = ktk.Player(markers, rigid_bodies, segments=segments, **viewing_options)
player.to_html5(start_time=0, stop_time=1)

For more information on kinematics, please check the [API Reference for the kinematics module](../api/kineticstoolkit.kinematics.rst).